# The empirical method for finding $Nu$

We want to use experimental data to find an expression for the Nusselt number, $Nu$ for a defined surface geometry. 

A simple experiment that can be perfomed is a flat plate which surface is kept at a uniform known constant temperature $T_s$ by an electric heating element with known power. If we minimize losses, we can obtain the rate of heat transferred to the fluid at the surface. Forced convection can be induced with a fluid flowing over it with known temperature $T_\infty$ and velocity $u_\infty$. The average convection coefficient $\bar{h}$ can be calculated by

$$\bar{h} = \frac{q''}{T_s - T_\infty}.$$

Knowing the characteristic length of the surface and the fluid properties, we can calculate the Nusselt ($Nu$), Reynolds ($Re$) and Prandtl ($Pr$) numbers using their definitions. Having the dimensionless number, we can obtain an expression that correlates them.

### How does this calculation look like?

Consider the following data for an experiment like the one described above. Four different fluids at 290 K were used with different velocity values. The plate surface was kept uniformly at 310 K and the heat flux $q''$ was obtained for each condition.




In [ ]:
import numpy as np

# characteristic length of the plate
L = 1     # in m

# Temperatures
Tinf = 290               # in K
Ts = 310                 # in K
Tf = (Tinf + Ts)/2       # in K

# Air
air_k = 26.3e-3    # at 300 K in W/mK
air_v = 15.89e-6   # at 300 K in m2/s
air_Pr = 0.707     # at 300 K
air_velocity = np.arange(0.1, 0.6, 0.1)                      # in m/s
air_heat = np.array([20.6, 38.0, 39.4, 56.8, 51.6])          # in W/m2

# Engine oil
oil_k = 145e-3     # at 300 K in W/mK
oil_v = 550e-6     # at 300 K in m2/s
oil_Pr = 6400      # at 300 K
oil_velocity = np.arange(1.0, 6.0, 1.0)                      # in m/s
oil_heat = np.array([1460., 2122., 2676., 2956., 3400.])     # in W/m2 

# Ethylene Glycol
glycol_k = 252e-3    # at 300 K in W/mK
glycol_v = 14.1e-6   # at 300 K in m2/s
glycol_Pr = 151      # at 300 K
glycol_velocity = np.arange(0.1, 0.6, 0.1)                   # in m/s
glycol_heat = np.array([1471., 2073., 2507., 2948., 3360.])  # in W/m2

# Water
water_k = 613e-3       # at 300 K in W/mK
water_v = 855e-6/997   # at 300 K in m2/s
water_Pr = 5.83        # at 300 K
water_velocity = np.arange(0.01, 0.06, 0.01)                 # in m/s
water_heat = np.array([1621., 2230., 2763., 3017., 3492.])   # in W/m2

We can now calculate the experimental convection coefficient for each data point

In [ ]:
# convection coefficient air
air_h = air_heat/(Ts - Tinf)
print('h_air =', air_h)

# convection coefficient engine oil
oil_h = 
print('h_oil =', oil_h)

# convection coefficient ethylene glycol
glycol_h = 


# convection coefficient water
water_h = 



and calculate the Nusselt number, $\bar{Nu_L} = \bar{h}L/k$, for each point as well. We can write a function for that

In [ ]:
def nusselt(h, k, L):
  

and pass the array with convection coefficients $h$ that we have just calculated.

In [ ]:
air_Nu = nusselt(air_h, air_k, L)
print('Nu_air =', air_Nu)
oil_Nu = nusselt(oil_h, oil_k, L)
print('Nu_oil =', oil_Nu)
glycol_Nu = nusselt(glycol_h, glycol_k, L)
print('Nu_glycol =', glycol_Nu)
water_Nu = nusselt(water_h, water_k, L)
print('Nu_water =', water_Nu)

We already have the Prandtl number, $Pr$, for each fluid (since its a property of the substance), so we need to calculate the Reynolds number, $Re_L = u_\infty L/\nu$.

In [ ]:
def reynolds():
  

air_Re = reynolds(air_velocity, L, air_v)
print('Re_air =', air_Re)
oil_Re = reynolds(oil_velocity, L, oil_v)
print('Re_oil =', oil_Re)
glycol_Re = reynolds(glycol_velocity, L, glycol_v)
print('Re_glycol =', glycol_Re)
water_Re = reynolds(water_velocity, L, water_v)
print('Re_water =', water_Re)

Now that we have all the information we need, how do we define a correlation for $Nu$, $Re$ and $Pr$?

Ideally, we need to know how the expression should look like so we can find the parameters. In this case, we know that the correlation has the form
$$ \bar{Nu_L} = C Re_L^m Pr^n. $$
So we can perfom a curve fitting with the data we have to find $C$, $m$, and $n$.

First, we need to organize the data in a way that we can use all of them at once. We will concatenate the four vectors we have for both $Re$ and $Nu$ into one big vector and create a big vector for $Pr$ of the same size.

In [ ]:
Re = np.concatenate((air_Re, oil_Re, glycol_Re, water_Re))
Nu = np.concatenate((air_Nu, oil_Nu, glycol_Nu, water_Nu))
Pr = np.concatenate((air_Pr*np.ones(5), oil_Pr*np.ones(5), glycol_Pr*np.ones(5), water_Pr*np.ones(5)))

Then we need to create an array with the dependent variables (the number of rows is the number of variables and the number of columns is how many data points we have) and an array with the independent variable (a vector of the size of the number of data points).

In [ ]:
xdata = np.concatenate((np.reshape(Re, (1,-1)),np.reshape(Pr, (1,-1))), axis = 0)    # arranging the independent variables Re (first row x[0]) and Pr (second row x[1]) in the right shape 
ydata = Nu    # defining the dependent variable Nu

We also need to define a function with the expression for the correlation with the variables and parameters ($C$, $m$, $n$) we want to find.

In [ ]:
def correlation(x, C, m, n):
  return C*x[0]**m*x[1]**n

We will now import the solver and perform the curve fitting.

In [ ]:
from scipy.optimize import curve_fit
par, cov = curve_fit(correlation, xdata, ydata)

print('C = ', par[0])
print('m = ', par[1])
print('n = ', par[2])


We can now plot the data and the correlation we obtained.

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(air_Re, air_Nu, label = 'Pr = %.3f' %(air_Pr))
ax.plot(air_Re, correlation(np.vstack((air_Re, air_Pr*np.ones(5))),par[0], par[1], par[2]))
ax.scatter(oil_Re, oil_Nu, label = 'Pr = %.0f' %(oil_Pr))
ax.plot(oil_Re, correlation(np.vstack((oil_Re, oil_Pr*np.ones(5))),par[0], par[1], par[2]))
ax.scatter(glycol_Re, glycol_Nu, label = 'Pr = %.0f' %(glycol_Pr))
ax.plot(glycol_Re, correlation(np.vstack((glycol_Re, glycol_Pr*np.ones(5))),par[0], par[1], par[2]))
ax.scatter(water_Re, water_Nu, label = 'Pr = %.2f' %(water_Pr))
ax.plot(water_Re, correlation(np.vstack((water_Re, water_Pr*np.ones(5))),par[0], par[1], par[2]))
ax.set_ylabel('Nu')
ax.set_xlabel('Re')
ax.legend()